### Creating the Question Database

In [337]:
import pandas as pd

from ast import literal_eval
from fpdf import FPDF

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

**Defining the Functions**

In [260]:
def add_question():
    add = True
    
    q = pd.read_csv('questions.csv', converters={'Category': literal_eval})    
    
    while add == True:
        
        max_ID = q['ID'].max()
        if pd.isna(max_ID):
            ID = 1
        else:
            ID = max_ID + 1

        cat = q['Category']
            
        cat = cat.explode().unique()
        print('Existing Categories: ', cat)
        
        Category = input('Question Category?')
        Category = Category.split('/')
    
        Topic = input('Specific Topic?')
    
        Difficulty = input('Difficulty? Easy, Medium or Hard')
    
        Question = input('Enter Trivia Question:')
    
        Correct = input('Correct Answer:')
    
        Choice1 = input('Incorrect Choice #1')
    
        Choice2 = input('Incorrect Choice #2')
        
        Choice3 = input('Incorrect Choice #3')
        
        Source = input('Input Source URL:')
    
        final_q = [ID, Category, Topic, Difficulty, Question, Correct, Choice1, Choice2, 
                   Choice3, Source]
        print(final_q)
    
        confirm = input('Does this look correct?')
    
        if confirm.lower() == 'y':
            q.loc[len(q)] = final_q
        
        another = input('Add another?')
        
        if another.lower() != 'y':
            add = False
    
    return q

In [178]:
def write_changes(q):
    q.to_csv('questions.csv', index=False)

**Adding Questions to Database**

In [690]:
q = add_question()

Existing Categories:  ['Halloween' 'Movies' 'DCOM' 'Disney' 'Marvel' 'Literature' 'Video Games'
 'Television' 'Sitcoms' 'Music' 'Pop Culture' 'Theme Parks']


Question Category? Television
Specific Topic? Star Trek
Difficulty? Easy, Medium or Hard Easy
Enter Trivia Question: What ship is famously captained by Captain James Kirk in the Star Trek series?
Correct Answer: USS Enterprise
Incorrect Choice #1 USS Truman
Incorrect Choice #2 USS Kennedy
Incorrect Choice #3 USS Marina
Input Source URL: https://en.wikipedia.org/wiki/USS_Enterprise_(NCC-1701)


[88, ['Television'], 'Star Trek', 'Easy', 'What ship is famously captained by Captain James Kirk in the Star Trek series?', 'USS Enterprise', 'USS Truman', 'USS Kennedy', 'USS Marina', 'https://en.wikipedia.org/wiki/USS_Enterprise_(NCC-1701)']


Does this look correct? y
Add another? n


In [691]:
write_changes(q)

**Category Breakdowns**

In [694]:
questions = pd.read_csv('questions.csv', converters={'Category': literal_eval})    

In [695]:
questions['Category'].explode().value_counts()

Movies         37
Television     25
Disney         16
Literature     13
Music          13
Video Games    12
Sitcoms        12
Pop Culture     9
Halloween       6
DCOM            3
Marvel          3
Theme Parks     1
Name: Category, dtype: int64

In [700]:
categories_count = questions['Category'].explode().value_counts().rename_axis('unique_values').to_frame('counts')
categories_count.to_csv('categories.csv', index=True)

**Generating Trivia**

In [495]:
category = 'General'
diff = 'Any'
q = 10

In [501]:
def generate(category, diff, q):
    
    trivia_db = pd.read_csv('questions.csv', converters={'Category': literal_eval})
    trivia_db['Topical'] = False

    if diff == 'Any':
        diff = ['Easy', 'Medium', 'Hard']
    
    for i, question in trivia_db.iterrows():
        if category == 'General':
            trivia_db = trivia_db.assign(Topical=True)
        elif (category in question['Category']) and (question['Difficulty'] in diff):
            trivia_db.at[i, 'Topical'] = True
            
    valid_qs = trivia_db.loc[trivia_db['Topical'] == True]
    
    if valid_qs.shape[0] < q:
        print('There are only', valid_qs.shape[0], 'questions that match your topic.', 
              valid_qs.shape[0], 'questions will be generated.')
        
    select = valid_qs.sample(n=q)
    select = select.reset_index()
    
    topics = []
    
    for i, question in trivia_db.iterrows():
        if question['Topic'] not in topics:
            topics.append(question['Topic'])
    
    create_pdf(select, category, topics)

In [502]:
def create_pdf(select, category, topics):
    pdf = FPDF()
    pdf.add_page()
    
    pg_w = pdf.w - 2*pdf.l_margin
    
    title = category + ' Trivia Night - Host Sheet'
    pdf.set_font('helvetica', 'B', size=20)
    pdf.cell(200, 10, txt=title, ln=1, align='C')
    
    topics = ', '.join(topics)
    tpc = 'Topics included: ' + topics
    pdf.set_font('helvetica', size=10)
    pdf.multi_cell(pg_w, 5, txt=tpc, ln=1, align='C')
    pdf.ln(5)

    for i, question in select.iterrows():
        ques = 'Question #' + str(i+1) + ': ' + question['Question']
        pdf.set_font('helvetica', 'B', size=14)
        pdf.multi_cell(pg_w, 7.5, txt=ques, ln=1, align='C')
        
        top_dif = 'Topic: ' + question['Topic'] + ' // Difficulty: ' + question['Difficulty']
        pdf.set_font('helvetica', size=10)
        pdf.cell(200, 5, txt=top_dif, ln=1, align='C')
        
        ans = 'Correct Answer: ' + question['Correct Answer']
        pdf.set_font('helvetica', 'B', size=12)
        pdf.cell(200, 5, txt=ans, ln=1, align='C')
        
        src = 'Source: ' + str(question['Source'])
        pdf.set_font('helvetica', size=10)
        pdf.multi_cell(pg_w, 5, txt=src, ln=1, align='C')
        pdf.ln(5)
        
    pdf.output('TriviaHostSheet.pdf')

**Defining User Selections**

In [503]:
trivia_db = pd.read_csv('questions.csv', converters={'Category': literal_eval})

In [504]:
categories = trivia_db['Category'].value_counts()
categories = categories[categories >= 10].index.tolist()
categories = categories[0]

topic_cat = trivia_db['Topic'].value_counts()
topic_cat = topics[topics >= 10].index.tolist()

try:
    categories.append(topic_cat[0])
except:
    pass

In [505]:
pdf = generate(category, diff, q)

**Difficulty**

In [512]:
for i, question in trivia_db.iterrows():
    if category == 'General':
        trivia_db = trivia_db.assign(Topical=True)
    elif (category in question['Category']) and (question['Difficulty'] in diff):
        trivia_db.at[i, 'Topical'] = True

In [513]:
valid_qs = trivia_db.loc[trivia_db['Topical'] == True]
valid_qs['Difficulty'].value_counts()

Medium    10
Easy       9
Hard       4
Name: Difficulty, dtype: int64

In [519]:
diff_opts = valid_qs['Difficulty'].value_counts()
diff_opts = diff_opts[diff_opts >= 10].index.tolist()
diff_opts = diff_opts
diff_opts.insert(0, 'Any')
diff_opts

['Any', 'Medium']

**Questions**

In [520]:
if diff == 'Any':
    diff = ['Easy', 'Medium', 'Hard']
else:
    diffic = []
    diffic.append(diff)
    diff = diffic

valid_qs = trivia_db.loc[trivia_db['Topical'] == True]
valid_qs = valid_qs.loc[trivia_db['Difficulty'].isin(diff)]

In [521]:
num_qs = valid_qs.shape[0]

max_qs = min(num_qs, 50)